# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,95.675083,-0.644112,1234,95.030971,96.319196,190.061942,192.638391
6,432.175794,0.589286,1234,432.765080,431.586508,865.530160,863.173017
7,558.778709,-0.962684,1234,557.816025,559.741393,1115.632051,1119.482786
8,921.992791,-0.109789,1234,921.883003,922.102580,1843.766006,1844.205160
9,218.952359,-0.103475,1234,218.848884,219.055835,437.697768,438.111669
10,947.420047,0.985480,1234,948.405527,946.434566,1896.811055,1892.869133
11,460.092096,0.447166,1234,460.539262,459.644930,921.078523,919.289860
12,986.626816,-0.218999,1234,986.407817,986.845816,1972.815634,1973.691632
13,556.993449,0.849047,1234,557.842495,556.144402,1115.684991,1112.288804
14,450.585663,0.095872,1234,450.681535,450.489791,901.363071,900.979582


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-95.030971,96.319196
6,-432.765080,431.586508
7,-557.816025,559.741393
8,-921.883003,922.102580
9,-218.848884,219.055835
10,-948.405527,946.434566
11,-460.539262,459.644930
12,-986.407817,986.845816
13,-557.842495,556.144402
14,-450.681535,450.489791


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-95.030971,96.319196
6,-432.765080,431.586508
7,-557.816025,559.741393
8,-921.883003,922.102580
9,-218.848884,219.055835
10,-948.405527,946.434566
11,-460.539262,459.644930
12,-986.407817,986.845816
13,-557.842495,556.144402
14,-450.681535,450.489791


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -95.03097112,   96.3191955 ],
       [-432.76508005,  431.58650842],
       [-557.81602543,  559.74139298],
       [-921.88300286,  922.1025801 ],
       [-218.84888424,  219.05583474],
       [-948.40552725,  946.43456644],
       [-460.53926166,  459.64492977],
       [-986.407817  ,  986.84581587],
       [-557.84249535,  556.14440188],
       [-450.68153538,  450.48979111]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,188.061942,-192.638391
6,863.530160,-863.173017
7,1113.632051,-1119.482786
8,1841.766006,-1844.205160
9,435.697768,-438.111669
10,1894.811055,-1892.869133
11,919.078523,-919.289860
12,1970.815634,-1973.691632
13,1113.684991,-1112.288804
14,899.363071,-900.979582


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
